# Lecture 7

## Links:
### https://www.youtube.com/watch?v=retAwpUv42E&list=PLOQjlWvnI0faRpH2oJcyW4CuM5Clt8a2n&index=7

## Walker's alias method
### Create a data structure to efficiently sample (O(1)) in O(n) preprocessing time
### Wikipedia's article implementation: https://en.wikipedia.org/wiki/Alias_method

In [1]:
import random

### Preprocessing
### O(n) time and memory

In [2]:
# check if x=y up to a tolerance 'tol'
def equal(x,y, tol=0.001):
    return abs(x-y) < tol

def preprocessing(distribution, debug=False):
    # 'From n buckets with p_i liquid, create n buckets with 1/n liquid on each of them'
    # input distribution is a list with the p_i's
    # output is a list U and a dict K. List U has n 'U_i's' entries s.t. bucket 0 has U_0
    # liquid on it (and maybe other liquid), bucket 1 has U_1 liquid on it (and maybe other liquid),
    # and so on. Dict K has 'K_i's' entries s.t. K[i] = j indicates that the other liquid on bucket i
    # is from bucket j
    # O(n) time and memory for preprocessing
    # We follow wikipedia's article implementation
    
    # initialize U_i=n*p_i. While doing this, divide the table entries into three categories:
    # overfull, underfull and exactly full
    U = []
    overfull = []
    underfull = []
    n = len(distribution)
    
    for i,pi in enumerate(distribution):
        Ui = n*pi
        U.append(Ui)
        
        if not(equal(Ui,1)):
            if Ui < 1:
                underfull.append(i)
            else:
                overfull.append(i)
        
    # while not all entries in output are 'exactly full'
    K = {}
    while overfull:
        if debug:
            print('---------------')
            print('U: ', U)
            print('K: ', K)
            print('overfull: ', overfull)
            print('underfull: ', underfull)
        
        # 1. Arbitrarily choose an overfull entry Ui > 1 and an underfull entry Uj < 1
        i = overfull.pop()
        j = underfull.pop()
        
        # 2. Allocate the unused space in entry j to outcome i, by setting Kj = i
        K[j] = i
        
        # 3. Remove the allocated space from entry i by changing Ui = Ui − (1 − Uj) = Ui + Uj − 1   
        U[i] -= 1-U[j]
        
        # 4. Entry j is now exactly full.
        
        # 5. Assign entry i to the appropriate category based on the new value of Ui
        if not(equal(U[i],1)):
            if U[i] < 1:
                underfull.append(i)
            else:
                overfull.append(i)
                    
    return U, K

### Test

In [3]:
# deterministic input
# n=5
d = [0.1, 0.03, 0.23, 0.52, 0.12]

print('distribution = ', d)

U, K = preprocessing(d, debug=True)

print('---------------')
print('U = ', U)
print('K = ', K)

distribution =  [0.1, 0.03, 0.23, 0.52, 0.12]
---------------
U:  [0.5, 0.15, 1.1500000000000001, 2.6, 0.6]
K:  {}
overfull:  [2, 3]
underfull:  [0, 1, 4]
---------------
U:  [0.5, 0.15, 1.1500000000000001, 2.2, 0.6]
K:  {4: 3}
overfull:  [2, 3]
underfull:  [0, 1]
---------------
U:  [0.5, 0.15, 1.1500000000000001, 1.35, 0.6]
K:  {4: 3, 1: 3}
overfull:  [2, 3]
underfull:  [0]
---------------
U:  [0.5, 0.15, 1.1500000000000001, 0.8500000000000001, 0.6]
K:  {4: 3, 1: 3, 0: 3}
overfull:  [2]
underfull:  [3]
---------------
U =  [0.5, 0.15, 1.0000000000000002, 0.8500000000000001, 0.6]
K =  {4: 3, 1: 3, 0: 3, 3: 2}


## Walker's alias method

### W Sampler
### Wikipedia's article implementation

In [4]:
def WSampler(distribution, k, debug=False):
    # Generate sample of size k from input distribution
    # O(n) time and memory for preprocessing,
    # O(k) time and memory for sampling k samples
    # Following wikipedia's article, we only generate one random number in [0,1)
    # and generate two random numbers from it
    
    print('---------------')
    print('---------------')
    print('Preprocessing...')
    U, K = preprocessing(distribution)
    print('Preprocessing done!')
        
    if debug:
        print('U = ', U)
        print('K = ', K)
    
    samples = []
    n = len(U)
    for _ in range(k):
        # 1. Generate a random number
        x = random.random()
        # 2a. Pick a bucket (uar in {0, 1, ..., n-1})
        i = int(n*x)
        # 2b. Pick a 'height' (uar in [0,1))
        y = (n*x)-i
        
        if debug:
            print('---------------')
            print('x: ', x)
            print('i: ', i)
            print('y: ', y)
        
        # 3
        if y < U[i]:
            samples.append(i)
        # 4
        else:
            samples.append(K[i])
        
    return samples

### test sampler

In [5]:
# deterministic input
# n=5, k=7, d = [0.1, 0.03, 0.23, 0.52, 0.12]
k = 7

# generate one sample of size k, to observe processing
# i.e., debug=True
print('sample:', WSampler(d, k, debug=True))
print('distribution:', d)

---------------
---------------
Preprocessing...
Preprocessing done!
U =  [0.5, 0.15, 1.0000000000000002, 0.8500000000000001, 0.6]
K =  {4: 3, 1: 3, 0: 3, 3: 2}
---------------
x:  0.9436000922903752
i:  4
y:  0.7180004614518758
---------------
x:  0.774368794632823
i:  3
y:  0.8718439731641152
---------------
x:  0.9173503409735692
i:  4
y:  0.5867517048678454
---------------
x:  0.906897597791648
i:  4
y:  0.5344879889582401
---------------
x:  0.48841356558269755
i:  2
y:  0.44206782791348775
---------------
x:  0.2587498403713322
i:  1
y:  0.293749201856661
---------------
x:  0.26397738283394845
i:  1
y:  0.31988691416974224
sample: [3, 2, 4, 4, 2, 3, 3]
distribution: [0.1, 0.03, 0.23, 0.52, 0.12]


In [6]:
# deterministic input
# n=5, k=7, d = [0.1, 0.03, 0.23, 0.52, 0.12]
k = 7
reps = 10

# generate sample of size k, reps times
for i in range(reps):
    print('---------------')
    print('---------------')
    print('---------------')
    print('iteration number:', i+1)
    print('sample:', WSampler(d, k))
    print('distribution:', d)

---------------
---------------
---------------
iteration number: 1
---------------
---------------
Preprocessing...
Preprocessing done!
sample: [2, 3, 3, 2, 3, 3, 2]
distribution: [0.1, 0.03, 0.23, 0.52, 0.12]
---------------
---------------
---------------
iteration number: 2
---------------
---------------
Preprocessing...
Preprocessing done!
sample: [3, 3, 2, 4, 2, 2, 2]
distribution: [0.1, 0.03, 0.23, 0.52, 0.12]
---------------
---------------
---------------
iteration number: 3
---------------
---------------
Preprocessing...
Preprocessing done!
sample: [4, 3, 2, 2, 3, 3, 4]
distribution: [0.1, 0.03, 0.23, 0.52, 0.12]
---------------
---------------
---------------
iteration number: 4
---------------
---------------
Preprocessing...
Preprocessing done!
sample: [3, 4, 0, 0, 4, 3, 2]
distribution: [0.1, 0.03, 0.23, 0.52, 0.12]
---------------
---------------
---------------
iteration number: 5
---------------
---------------
Preprocessing...
Preprocessing done!
sample: [3, 0, 2, 3